In [ ]:
import torch
print(f"PyTorch Version: {torch.__version__}")
print(f"CUDA Available: {torch.cuda.is_available()}")
print(f"GPU Count: {torch.cuda.device_count()}")

try:
    import flash_attn
    print("✅ Flash Attention 2 installed successfully!")
except ImportError:
    print("❌ Flash Attention 2 NOT found.")

try:
    import vllm
    print("✅ vLLM installed successfully!")
except ImportError:
    print("❌ vLLM NOT found.")

# PyTorch Version: 2.5.1+cu124
# CUDA Available: True
# GPU Count: 4
# ✅ Flash Attention 2 installed successfully!
# INFO 12-07 13:41:06 __init__.py:190] Automatically detected platform cuda.
# ✅ vLLM installed successfully!

PyTorch Version: 2.5.1+cu124
CUDA Available: True
GPU Count: 4
✅ Flash Attention 2 installed successfully!
INFO 12-07 13:41:06 __init__.py:190] Automatically detected platform cuda.
✅ vLLM installed successfully!


In [3]:
import json
import os

# 复用第一步的路径
DATA_PATH = "data/MATH/validation.jsonl"

# --- 1. 定义 R1-Zero 的模版 (这就是“印制试卷”的模具) ---
# 参考作业文档 Page 4 的 r1_zero prompt [cite: 78]
def apply_r1_zero_template(raw_question):
    """
    输入：原始题目文本 (例如 "1+1=?")
    输出：包装好的 Prompt 字符串
    """
    template = (
        "A conversation between User and Assistant. The User asks a question, and the Assistant solves it. "
        "The Assistant first thinks about the reasoning process in the mind and then provides the User with the answer. "
        "The reasoning process is enclosed within <think> </think> and answer is enclosed within <answer> </answer> tags, respectively, "
        "i.e., <think> reasoning process here </think> <answer> answer here </answer>.\n"
        f"User: {raw_question}\n"
        "Assistant: <think>"  # <--- 关键点：这一行逼着模型开始思考 [cite: 82]
    )
    return template

# --- 2. 加载数据的简易版 (复用上一步逻辑) ---
def load_raw_questions(filepath):
    qs = []
    if os.path.exists(filepath):
        with open(filepath, "r", encoding="utf-8") as f:
            for line in f:
                qs.append(json.loads(line)["question"])
    return qs

# --- 主程序 ---
if __name__ == "__main__":
    # 1. 先把题目拿出来
    raw_questions = load_raw_questions(DATA_PATH)
    print(f"📚 拿到 {len(raw_questions)} 道原始题目。")

    # 2. 开始“印卷子” (批量处理)
    formatted_prompts = []
    print("🖨️  正在印制 R1-Zero 考卷...")
    
    for q in raw_questions:
        # 每一个题目都塞进模具里
        prompt = apply_r1_zero_template(q)
        formatted_prompts.append(prompt)

    # --- 3. 检查成果 (Visual Check) ---
    print("\n" + "="*40)
    print("🔎 查看第一张印好的考卷 (Sample 0):")
    print("-" * 20)
    # 打印出来看看，你会发现它比原始题目长了很多
    print(formatted_prompts[0]) 
    print("="*40)
    
    # 验证一下结尾是不是我们想要的
    if formatted_prompts[0].endswith("Assistant: <think>"):
        print("✅ 格式检查通过！试卷末尾成功强制了 '<think>'。")
    else:
        print("❌ 格式错误！末尾没有诱导标签。")

📚 拿到 5000 道原始题目。
🖨️  正在印制 R1-Zero 考卷...

🔎 查看第一张印好的考卷 (Sample 0):
--------------------
A conversation between User and Assistant. The User asks a question, and the Assistant solves it. The Assistant first thinks about the reasoning process in the mind and then provides the User with the answer. The reasoning process is enclosed within <think> </think> and answer is enclosed within <answer> </answer> tags, respectively, i.e., <think> reasoning process here </think> <answer> answer here </answer>.
User: How many vertical asymptotes does the graph of $y=\frac{2}{x^2+x-6}$ have?
Assistant: <think>
✅ 格式检查通过！试卷末尾成功强制了 '<think>'。
